In [84]:
import boto3
import os
import json
import time
from datetime import datetime
import uuid

sts_client = boto3.client('sts')
session = boto3.session.Session()

account_id = sts_client.get_caller_identity()["Account"]
region = session.region_name

s3_client = boto3.client('s3', region)
bedrock_client = boto3.client('bedrock-runtime', region)

agent_foundation_model = [
    'us.anthropic.claude-3-5-haiku-20241022-v1:0'
]

## Importing helper functions
On following section, we're adding bedrock_agent_helper.py and knowledge_base_helper on Python path, so the files can be recognized and their functionalities can be invoked.

In general, the helper functions handle common tasks including agent creation, knowledge bases for bedrock creation and accessing data on S3.

In [2]:
import sys

sys.path.insert(0, ".")
sys.path.insert(1, "..")

from utils.bedrock_agent_helper import (
    AgentsForAmazonBedrock
)
from utils.knowledge_base_helper import (
    KnowledgeBasesForAmazonBedrock
)
agents = AgentsForAmazonBedrock()
kb = KnowledgeBasesForAmazonBedrock()

## Create and syncronize Knowledge Base
Before creating an agent, we need to create a Knowledge Base and associates it with the PR generator agent.
This KB will contain some prestine examples of high quality PRs across different movies. We have synthetically generated some example of PRs and stored in the `good_prs` folder. We'll use them as the basis to create our knowledge base system.

This creation process can take several minutes.

In [3]:
knowledge_base_name = f'lab5-media-agent-kb'
knowledge_base_description = "KB containing information about media dataset"
s3_bucket_name = f"labs-bucket-{region}-{account_id}"
bucket_prefix = "lab5/"

In [4]:
%%time
kb_id, ds_id = kb.create_or_retrieve_knowledge_base(
    knowledge_base_name,
    knowledge_base_description,
    s3_bucket_name,
    "amazon.titan-embed-text-v2:0",
    bucket_prefix
)

print(f"Knowledge Base ID: {kb_id}")
print(f"Data Source ID: {ds_id}")

Creating KB lab5-media-agent-kb
Step 1 - Creating or retrieving labs-bucket-us-east-1-338146311978 S3 bucket for Knowledge Base documents
Bucket labs-bucket-us-east-1-338146311978 already exists - retrieving it!
Step 2 - Creating Knowledge Base Execution Role (AmazonBedrockExecutionRoleForKnowledgeBase_457) and Policies
Step 3 - Creating OSS encryption, network and data access policies
Step 4 - Creating OSS Collection (this step takes a couple of minutes to complete)
{ 'ResponseMetadata': { 'HTTPHeaders': { 'connection': 'keep-alive',
                                         'content-length': '315',
                                         'content-type': 'application/x-amz-json-1.0',
                                         'date': 'Fri, 09 May 2025 05:03:57 '
                                                 'GMT',
                                         'x-amzn-requestid': '0c845084-0bd9-4dc0-adad-e94c3069aba6'},
                        'HTTPStatusCode': 200,
                       

In [22]:
kb.synchronize_data(kb_id, ds_id)

{ 'dataSourceId': 'AFL3PEL1TJ',
  'ingestionJobId': 'SL6JXYXAWU',
  'knowledgeBaseId': 'NDVODGINTK',
  'startedAt': datetime.datetime(2025, 5, 9, 5, 15, 39, 48473, tzinfo=tzlocal()),
  'statistics': { 'numberOfDocumentsDeleted': 0,
                  'numberOfDocumentsFailed': 0,
                  'numberOfDocumentsScanned': 0,
                  'numberOfMetadataDocumentsModified': 0,
                  'numberOfMetadataDocumentsScanned': 0,
                  'numberOfModifiedDocumentsIndexed': 0,
                  'numberOfNewDocumentsIndexed': 0},
  'status': 'STARTING',
  'updatedAt': datetime.datetime(2025, 5, 9, 5, 15, 39, 48473, tzinfo=tzlocal())}
{ 'dataSourceId': 'AFL3PEL1TJ',
  'ingestionJobId': 'SL6JXYXAWU',
  'knowledgeBaseId': 'NDVODGINTK',
  'startedAt': datetime.datetime(2025, 5, 9, 5, 15, 39, 48473, tzinfo=tzlocal()),
  'statistics': { 'numberOfDocumentsDeleted': 0,
                  'numberOfDocumentsFailed': 0,
                  'numberOfDocumentsScanned': 25,
          

# Creating an Agent

In [81]:
kb_info = kb.get_kb(kb_id)
kb_arn = kb_info['knowledgeBase']['knowledgeBaseArn']

In [100]:
kb_config = {
    'kb_id': kb_id,
    'kb_instruction': """This knowledge base contains information about media titles. Use this knowledge base to find information about the media, including the title and id."""
}

In [105]:
agent_description = """A media agent that helps users find media information."""
agent_instruction = """You are a professional media agent. Your task is to help users find the information
related to media based on their questions.

You have access to a knowledge base and tools that provide information media details, including the title, id.
You also have access to tools that can help provide additional information about the title, such as rating and others.
"""

In [106]:
agent_suffix = str(uuid.uuid4())[:5]
agent_name = f"lab5-media_agent-{agent_suffix}"

media_agent = agents.create_agent(
    agent_name,
    agent_description,
    agent_instruction,
    agent_foundation_model,
    kb_arns=[kb_arn],
    code_interpretation=False
)

media_agent

('N3EDCYHNUG',
 'TSTALIASID',
 'arn:aws:bedrock:us-east-1:338146311978:agent-alias/N3EDCYHNUG/TSTALIASID')

In [107]:
agents.associate_kb_with_agent(
    media_agent[0], # the agent ID
    kb_config['kb_instruction'],
    kb_config['kb_id']
)

In [108]:
action_group_name=f"get-movie-ratings-{agent_suffix}"
action_group_descr = "retrieves movie rating for a given title id"
lambda_code = "lambda_handler.py"
function_defs = [
            {
                'description': 'retrieves media ratings',
                'name': 'get-media-ratings',
                'parameters': {
                    'title_id': {
                        'description': 'The ID of the media title',
                        'required': True,
                        'type': 'string'
                    }
                },
                'requireConfirmation': 'DISABLED'
            },
        ]

In [109]:
lambda_function_name = f"lab5-get-movie-ratings-{agent_suffix}"
action_group_arn = agents.add_action_group_with_lambda(agent_name,
                                         lambda_function_name, lambda_code, 
                                         function_defs, action_group_name, action_group_descr)

In [110]:
%%time
response = agents.invoke(
    """What is the rating for "architecting with AWS"? 
""", 
    media_agent[0], enable_trace=True,
    trace_level = "all"
)
print("====================")
print(response)

invokeAgent API response object: {'ResponseMetadata': {'RequestId': 'edf445b0-0fc0-4498-a2b4-0fb21e3ec92a', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Fri, 09 May 2025 07:11:42 GMT', 'content-type': 'application/vnd.amazon.eventstream', 'transfer-encoding': 'chunked', 'connection': 'keep-alive', 'x-amzn-requestid': 'edf445b0-0fc0-4498-a2b4-0fb21e3ec92a', 'x-amz-bedrock-agent-session-id': '00d8331a-2c93-11f0-89ae-3e83cdaeea4f', 'x-amzn-bedrock-agent-content-type': 'application/json'}, 'RetryAttempts': 0}, 'contentType': 'application/json', 'sessionId': '00d8331a-2c93-11f0-89ae-3e83cdaeea4f', 'completion': <botocore.eventstream.EventStream object at 0x7f6d8880a090>}
---
{
  "agentAliasId": "TSTALIASID",
  "agentId": "N3EDCYHNUG",
  "agentVersion": "DRAFT",
  "callerChain": [
    {
      "agentAliasArn": "arn:aws:bedrock:us-east-1:338146311978:agent-alias/N3EDCYHNUG/TSTALIASID"
    }
  ],
  "eventTime": "2025-05-09 07:11:42.344632+00:00",
  "sessionId": "00d8331a-2c93-11f0-89ae-3e83c